# Prereq

In [94]:
!python3 -m pip install --upgrade pip
!apt install autodock-vina git -y
!pip install plip

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
autodock-vina is already the newest version (1.2.3-2).
git is already the newest version (1:2.34.1-1ubuntu1.15).
0 upgraded, 0 newly installed, 0 to remove and 130 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.1/35.1 MB 3.2 MB/s  0:00:10m0:00:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 3.6 MB/s  0:00:00 eta 0:00:01m
  Created wheel for prody: filename=prody-2.6.1-cp311-cp311-linux_x86_64.whl size=36499006 sha256=2add2d8148f1090d41f5bd66aa24b15c1ef48555656a1e956a8a55915c78f7da
  Stored in directory: /root/.cache/pip/wheels/c1/f0/d2/42b08ebde1b316d1b7477373fe926adb297d9635b14ae7e711
Successfully built prody
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [prody]32m1/2 [prody]


# Ligand Docking

## HOLO

### Enniatin A

In [146]:
!vina --receptor HSA_Target/2BXD/2bxd_clean.pdbqt \
      --ligand Ligand_Pubchem_Data/EnniantinA/EnniatinA.pdbqt \
      --config dockprep/2bxd-site1.txt \
      --scoring vinardo \
      --exhaustiveness=32 --out Results/Site1/HOLO/ENA-Site1.pdbqt

AutoDock Vina v1.2.3
#################################################################
# If you used AutoDock Vina in your work, please cite:          #
#                                                               #
# J. Eberhardt, D. Santos-Martins, A. F. Tillack, and S. Forli  #
# AutoDock Vina 1.2.0: New Docking Methods, Expanded Force      #
# Field, and Python Bindings, J. Chem. Inf. Model. (2021)       #
# DOI 10.1021/acs.jcim.1c00203                                  #
#                                                               #
# O. Trott, A. J. Olson,                                        #
# AutoDock Vina: improving the speed and accuracy of docking    #
# with a new scoring function, efficient optimization and       #
# multithreading, J. Comp. Chem. (2010)                         #
# DOI 10.1002/jcc.21334                                         #
#                                                               #
# Please see https://github.com/ccsb-scripps/AutoDock-V

In [117]:
!obabel Results/Site1/HOLO/ENA-Site1.pdbqt -O Results/Site1/HOLO/ENA-Site1_.pdb -m

8 molecules converted
8 files output. The first is Results/Site1/HOLO/ENA-Site1_1.pdb


In [145]:
!vina --receptor HSA_Target/2BXG/2bxg_clean.pdbqt \
      --ligand Ligand_Pubchem_Data/EnniantinA/EnniatinA.pdbqt \
      --config dockprep/2bxg-site2.txt \
      --scoring vinardo \
      --exhaustiveness=32 --out Results/Site2/HOLO/ENA-Site2.pdbqt

AutoDock Vina v1.2.3
#################################################################
# If you used AutoDock Vina in your work, please cite:          #
#                                                               #
# J. Eberhardt, D. Santos-Martins, A. F. Tillack, and S. Forli  #
# AutoDock Vina 1.2.0: New Docking Methods, Expanded Force      #
# Field, and Python Bindings, J. Chem. Inf. Model. (2021)       #
# DOI 10.1021/acs.jcim.1c00203                                  #
#                                                               #
# O. Trott, A. J. Olson,                                        #
# AutoDock Vina: improving the speed and accuracy of docking    #
# with a new scoring function, efficient optimization and       #
# multithreading, J. Comp. Chem. (2010)                         #
# DOI 10.1002/jcc.21334                                         #
#                                                               #
# Please see https://github.com/ccsb-scripps/AutoDock-V

In [44]:
!obabel Results/Site2/HOLO/ENA-Site2.pdbqt -O Results/Site2/HOLO/ENA-Site2_.pdb -m

12 molecules converted
12 files output. The first is Results/ENA-Site2/ENA-Site2_1.pdb


### Enniatin A1

In [120]:
!vina --receptor HSA_Target/2BXD/2bxd_clean.pdbqt \
      --ligand Ligand_Pubchem_Data/EnniantinA1/EnniatinA1.pdbqt \
      --config dockprep/2bxd-site1.txt \
      --scoring vinardo \
      --exhaustiveness=32 --out Results/Site1/HOLO/ENA1-Site1.pdbqt

AutoDock Vina v1.2.3
#################################################################
# If you used AutoDock Vina in your work, please cite:          #
#                                                               #
# J. Eberhardt, D. Santos-Martins, A. F. Tillack, and S. Forli  #
# AutoDock Vina 1.2.0: New Docking Methods, Expanded Force      #
# Field, and Python Bindings, J. Chem. Inf. Model. (2021)       #
# DOI 10.1021/acs.jcim.1c00203                                  #
#                                                               #
# O. Trott, A. J. Olson,                                        #
# AutoDock Vina: improving the speed and accuracy of docking    #
# with a new scoring function, efficient optimization and       #
# multithreading, J. Comp. Chem. (2010)                         #
# DOI 10.1002/jcc.21334                                         #
#                                                               #
# Please see https://github.com/ccsb-scripps/AutoDock-V

In [121]:
!obabel Results/Site1/HOLO/ENA1-Site1.pdbqt -O Results/Site1/HOLO/ENA1-Site1_.pdb -m

9 molecules converted
9 files output. The first is Results/Site1/HOLO/ENA1-Site1_1.pdb


In [122]:
!vina --receptor HSA_Target/2BXG/2bxg_clean.pdbqt \
      --ligand Ligand_Pubchem_Data/EnniantinA1/EnniatinA1.pdbqt \
      --config dockprep/2bxg-site2.txt \
      --scoring vinardo \
      --exhaustiveness=32 --out Results/Site2/HOLO/ENA1-Site2.pdbqt

AutoDock Vina v1.2.3
#################################################################
# If you used AutoDock Vina in your work, please cite:          #
#                                                               #
# J. Eberhardt, D. Santos-Martins, A. F. Tillack, and S. Forli  #
# AutoDock Vina 1.2.0: New Docking Methods, Expanded Force      #
# Field, and Python Bindings, J. Chem. Inf. Model. (2021)       #
# DOI 10.1021/acs.jcim.1c00203                                  #
#                                                               #
# O. Trott, A. J. Olson,                                        #
# AutoDock Vina: improving the speed and accuracy of docking    #
# with a new scoring function, efficient optimization and       #
# multithreading, J. Comp. Chem. (2010)                         #
# DOI 10.1002/jcc.21334                                         #
#                                                               #
# Please see https://github.com/ccsb-scripps/AutoDock-V

In [123]:
!obabel Results/Site2/HOLO/ENA1-Site2.pdbqt -O Results/Site2/HOLO/ENA1-Site2_.pdb -m

7 molecules converted
7 files output. The first is Results/Site2/HOLO/ENA1-Site2_1.pdb


### Enniantin B

In [124]:
!vina --receptor HSA_Target/2BXD/2bxd_clean.pdbqt \
      --ligand Ligand_Pubchem_Data/EnniantinB/EnniatinB.pdbqt \
      --config dockprep/2bxd-site1.txt \
      --scoring vinardo \
      --exhaustiveness=32 --out Results/Site1/HOLO/ENB-Site1.pdbqt

AutoDock Vina v1.2.3
#################################################################
# If you used AutoDock Vina in your work, please cite:          #
#                                                               #
# J. Eberhardt, D. Santos-Martins, A. F. Tillack, and S. Forli  #
# AutoDock Vina 1.2.0: New Docking Methods, Expanded Force      #
# Field, and Python Bindings, J. Chem. Inf. Model. (2021)       #
# DOI 10.1021/acs.jcim.1c00203                                  #
#                                                               #
# O. Trott, A. J. Olson,                                        #
# AutoDock Vina: improving the speed and accuracy of docking    #
# with a new scoring function, efficient optimization and       #
# multithreading, J. Comp. Chem. (2010)                         #
# DOI 10.1002/jcc.21334                                         #
#                                                               #
# Please see https://github.com/ccsb-scripps/AutoDock-V

In [144]:
!obabel Results/Site1/HOLO/ENB-Site1.pdbqt -O Results/Site1/HOLO/ENB-Site1_.pdb -m

2 molecules converted
2 files output. The first is Results/Site1/HOLO/ENB-Site1_1.pdb


In [126]:
!vina --receptor HSA_Target/2BXG/2bxg_clean.pdbqt \
      --ligand Ligand_Pubchem_Data/EnniantinB/EnniatinB.pdbqt \
      --config dockprep/2bxg-site2.txt \
      --scoring vinardo \
      --exhaustiveness=32 --out Results/Site2/HOLO/ENB-Site2.pdbqt

AutoDock Vina v1.2.3
#################################################################
# If you used AutoDock Vina in your work, please cite:          #
#                                                               #
# J. Eberhardt, D. Santos-Martins, A. F. Tillack, and S. Forli  #
# AutoDock Vina 1.2.0: New Docking Methods, Expanded Force      #
# Field, and Python Bindings, J. Chem. Inf. Model. (2021)       #
# DOI 10.1021/acs.jcim.1c00203                                  #
#                                                               #
# O. Trott, A. J. Olson,                                        #
# AutoDock Vina: improving the speed and accuracy of docking    #
# with a new scoring function, efficient optimization and       #
# multithreading, J. Comp. Chem. (2010)                         #
# DOI 10.1002/jcc.21334                                         #
#                                                               #
# Please see https://github.com/ccsb-scripps/AutoDock-V

In [127]:
!obabel Results/Site2/HOLO/ENB-Site2.pdbqt -O Results/Site2/HOLO/ENB-Site2_.pdb -m

11 molecules converted
11 files output. The first is Results/Site2/HOLO/ENB-Site2_1.pdb


### Eniantin B1

In [136]:
!vina --receptor HSA_Target/2BXD/2bxd_clean.pdbqt \
      --ligand Ligand_Pubchem_Data/EnniantinB1/EnniatinB1.pdbqt \
      --config dockprep/2bxd-site1.txt \
      --scoring vinardo \
      --exhaustiveness=32 --out Results/Site1/HOLO/ENB1-Site1.pdbqt

AutoDock Vina v1.2.3
#################################################################
# If you used AutoDock Vina in your work, please cite:          #
#                                                               #
# J. Eberhardt, D. Santos-Martins, A. F. Tillack, and S. Forli  #
# AutoDock Vina 1.2.0: New Docking Methods, Expanded Force      #
# Field, and Python Bindings, J. Chem. Inf. Model. (2021)       #
# DOI 10.1021/acs.jcim.1c00203                                  #
#                                                               #
# O. Trott, A. J. Olson,                                        #
# AutoDock Vina: improving the speed and accuracy of docking    #
# with a new scoring function, efficient optimization and       #
# multithreading, J. Comp. Chem. (2010)                         #
# DOI 10.1002/jcc.21334                                         #
#                                                               #
# Please see https://github.com/ccsb-scripps/AutoDock-V

In [142]:
!obabel Results/Site1/HOLO/ENB-Site1.pdbqt -O Results/Site1/HOLO/ENB-Site1_.pdb -m

2 molecules converted
2 files output. The first is Results/Site1/HOLO/ENB-Site1_1.pdb


In [137]:
!vina --receptor HSA_Target/2BXG/2bxg_clean.pdbqt \
      --ligand Ligand_Pubchem_Data/EnniantinB1/EnniatinB1.pdbqt \
      --config dockprep/2bxg-site2.txt \
      --scoring vinardo \
      --exhaustiveness=32 --out Results/Site2/HOLO/ENB1-Site2.pdbqt

AutoDock Vina v1.2.3
#################################################################
# If you used AutoDock Vina in your work, please cite:          #
#                                                               #
# J. Eberhardt, D. Santos-Martins, A. F. Tillack, and S. Forli  #
# AutoDock Vina 1.2.0: New Docking Methods, Expanded Force      #
# Field, and Python Bindings, J. Chem. Inf. Model. (2021)       #
# DOI 10.1021/acs.jcim.1c00203                                  #
#                                                               #
# O. Trott, A. J. Olson,                                        #
# AutoDock Vina: improving the speed and accuracy of docking    #
# with a new scoring function, efficient optimization and       #
# multithreading, J. Comp. Chem. (2010)                         #
# DOI 10.1002/jcc.21334                                         #
#                                                               #
# Please see https://github.com/ccsb-scripps/AutoDock-V

In [143]:
!obabel Results/Site2/HOLO/ENB1-Site2.pdbqt -O Results/Site2/HOLO/ENB1-Site2_.pdb -m

14 molecules converted
14 files output. The first is Results/Site2/HOLO/ENB1-Site2_1.pdb


### Warfarin

In [128]:
!vina --receptor HSA_Target/2BXD/2bxd_clean.pdbqt \
      --ligand RCSB_Ligand/Warfarin/RWF.pdbqt \
      --config dockprep/2bxd-site1.txt \
      --scoring vinardo \
      --exhaustiveness=32 --out Results/Site1/HOLO/RWF-Site1.pdbqt

AutoDock Vina v1.2.3
#################################################################
# If you used AutoDock Vina in your work, please cite:          #
#                                                               #
# J. Eberhardt, D. Santos-Martins, A. F. Tillack, and S. Forli  #
# AutoDock Vina 1.2.0: New Docking Methods, Expanded Force      #
# Field, and Python Bindings, J. Chem. Inf. Model. (2021)       #
# DOI 10.1021/acs.jcim.1c00203                                  #
#                                                               #
# O. Trott, A. J. Olson,                                        #
# AutoDock Vina: improving the speed and accuracy of docking    #
# with a new scoring function, efficient optimization and       #
# multithreading, J. Comp. Chem. (2010)                         #
# DOI 10.1002/jcc.21334                                         #
#                                                               #
# Please see https://github.com/ccsb-scripps/AutoDock-V

In [140]:
!obabel Results/Site1/HOLO/RWF-Site1.pdbqt -O Results/Site1/HOLO/RWF-Site1_.pdb -m

20 molecules converted
20 files output. The first is Results/Site1/HOLO/RWF-Site1_1.pdb


In [138]:
!vina --receptor HSA_Target/2BXG/2bxg_clean.pdbqt \
      --ligand RCSB_Ligand/Warfarin/RWF.pdbqt \
      --config dockprep/2bxg-site2.txt \
      --scoring vinardo \
      --exhaustiveness=32 --out Results/Site2/HOLO/RWF-Site2.pdbqt

AutoDock Vina v1.2.3
#################################################################
# If you used AutoDock Vina in your work, please cite:          #
#                                                               #
# J. Eberhardt, D. Santos-Martins, A. F. Tillack, and S. Forli  #
# AutoDock Vina 1.2.0: New Docking Methods, Expanded Force      #
# Field, and Python Bindings, J. Chem. Inf. Model. (2021)       #
# DOI 10.1021/acs.jcim.1c00203                                  #
#                                                               #
# O. Trott, A. J. Olson,                                        #
# AutoDock Vina: improving the speed and accuracy of docking    #
# with a new scoring function, efficient optimization and       #
# multithreading, J. Comp. Chem. (2010)                         #
# DOI 10.1002/jcc.21334                                         #
#                                                               #
# Please see https://github.com/ccsb-scripps/AutoDock-V

In [141]:
!obabel Results/Site2/HOLO/RWF-Site2.pdbqt -O Results/Site2/HOLO/RWF-Site2_.pdb -m

20 molecules converted
20 files output. The first is Results/Site2/HOLO/RWF-Site2_1.pdb


### Ibuprofen

In [135]:
!vina --receptor HSA_Target/2BXD/2bxd_clean.pdbqt \
      --ligand RCSB_Ligand/Ibuprofen/IBP.pdbqt \
      --config dockprep/2bxd-site1.txt \
      --scoring vinardo \
      --exhaustiveness=32 --out Results/Site1/HOLO/IBP-Site1.pdbqt

AutoDock Vina v1.2.3
#################################################################
# If you used AutoDock Vina in your work, please cite:          #
#                                                               #
# J. Eberhardt, D. Santos-Martins, A. F. Tillack, and S. Forli  #
# AutoDock Vina 1.2.0: New Docking Methods, Expanded Force      #
# Field, and Python Bindings, J. Chem. Inf. Model. (2021)       #
# DOI 10.1021/acs.jcim.1c00203                                  #
#                                                               #
# O. Trott, A. J. Olson,                                        #
# AutoDock Vina: improving the speed and accuracy of docking    #
# with a new scoring function, efficient optimization and       #
# multithreading, J. Comp. Chem. (2010)                         #
# DOI 10.1002/jcc.21334                                         #
#                                                               #
# Please see https://github.com/ccsb-scripps/AutoDock-V

In [134]:
!vina --receptor HSA_Target/2BXG/2bxg_clean.pdbqt \
      --ligand RCSB_Ligand/Ibuprofen/IBP.pdbqt \
      --config dockprep/2bxg-site2.txt \
      --scoring vinardo \
      --exhaustiveness=32 --out Results/Site2/HOLO/IBP-Site2.pdbqt

AutoDock Vina v1.2.3
#################################################################
# If you used AutoDock Vina in your work, please cite:          #
#                                                               #
# J. Eberhardt, D. Santos-Martins, A. F. Tillack, and S. Forli  #
# AutoDock Vina 1.2.0: New Docking Methods, Expanded Force      #
# Field, and Python Bindings, J. Chem. Inf. Model. (2021)       #
# DOI 10.1021/acs.jcim.1c00203                                  #
#                                                               #
# O. Trott, A. J. Olson,                                        #
# AutoDock Vina: improving the speed and accuracy of docking    #
# with a new scoring function, efficient optimization and       #
# multithreading, J. Comp. Chem. (2010)                         #
# DOI 10.1002/jcc.21334                                         #
#                                                               #
# Please see https://github.com/ccsb-scripps/AutoDock-V

## APO

# PLIP

In [ ]:
!plip -f Results/4l9k-CC-Site1/complex_4l9k_cc_site1_1.pdb -o results/4l9k-CC-Site1/

In [15]:
!plip -f Results/4l9k-CC-Site1/complex_4l9k_cc_site1_1.pdb -o results/4l9k-CC-Site1/

None
2025-08-26 11:46:37,713 [INFO] [plipcmd.py:134] plip.plipcmd: Protein-Ligand Interaction Profiler (PLIP) 2.4.0
2025-08-26 11:46:37,713 [INFO] [plipcmd.py:135] plip.plipcmd: brought to you by: PharmAI GmbH (2020-2021) - www.pharm.ai - hello@pharm.ai
2025-08-26 11:46:37,714 [INFO] [plipcmd.py:136] plip.plipcmd: please cite: Adasme,M. et al. PLIP 2021: expanding the scope of the protein-ligand interaction profiler to DNA and RNA. Nucl. Acids Res. (05 May 2021), gkab294. doi: 10.1093/nar/gkab294
2025-08-26 11:46:37,715 [INFO] [plipcmd.py:60] plip.plipcmd: starting analysis of complex_4l9k_cc_site1_1.pdb
2025-08-26 11:46:38,672 [INFO] [plipcmd.py:175] plip.plipcmd: finished analysis, find the result files in results/4l9k-CC-Site1/
